In [ ]:
# !pip3 install --upgrade pip
# !pip3 install --upgrade wandb nsepy keras pandas sklearn imageio pprint mplcursors tqdm livelossplot keras-tqdm

In [1]:
import warnings; warnings.simplefilter('ignore')
import ssl
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from urllib3.exceptions import ProtocolError
from ssl import SSLError
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Input,Flatten,Activation,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard 
import tensorflow.keras
import matplotlib.pyplot as plt
import nsepy
import pprint
import os
from datetime import date,timedelta
import datetime
import mplcursors
plt.style.use('fivethirtyeight')
from tqdm import tqdm
import pickle
from sklearn import preprocessing
import livelossplot
import wandb
# from wandb.keras import WandbCallback
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from IPython.display import clear_output
import seaborn as sns
homediraddr = '/home/hemangjoshi37a_gmail_com'

Using TensorFlow backend.


In [ ]:
# os.listdir('/')

In [ ]:
wandb.init(project="test",magic=True,sync_tensorboard=True) 

In [5]:
# datahj={}
# with open('/home/hemangjoshi37a_gmail_com/data.p', 'rb') as fp:
#     datahj = pickle.load(fp)

# #Remove empty things
# removables=[]
# for onename in datahj.keys():
#     nparrayy=np.array(datahj[onename])
#     if(nparrayy.size==0):
#         removables.append(onename)
        
# for oneremovename in tqdm(removables):
#     del datahj[oneremovename]
    
with open('x_train.p', 'rb') as fp:
    x_train = pickle.load(fp)

with open('y_train.p', 'rb') as fp:
    y_train = pickle.load(fp)
    
with open('scale_dict.p', 'rb') as fp:
    scale_dict = pickle.load(fp)
    
with open(homediraddr+'/pfiles'+'/TATASTEEL_DF.p', 'rb') as fp:
    TATASTEEL_DF = pickle.load(fp)
    
# model=load_model('current34.h5')

In [6]:
# stock_name='TATASTEEL'
# df = nsepy.get_history(symbol=stock_name, start=date(2012,1,1), end=date.today())

# df=datahj['TATASTEEL']
df=TATASTEEL_DF

data = df.filter(['close'])  #old_line

scaler=scale_dict['TATASTEEL']

dataset = data.values
training_data_len = math.ceil( len(dataset) * .97 )

# scaler = MinMaxScaler(feature_range=(0,1))
# scaler=scaler.fit(dataset)
scaled_data = scaler.transform(dataset)

x_train = []
y_train = []
train_data = scaled_data[0:training_data_len , :]
for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, :])
  y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, :])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
print('x_test shape : ',x_test.shape)
print('x_train shape : ',x_train.shape)

x_test shape :  (14711, 60, 1)
x_train shape :  (475613, 60, 1)


In [ ]:
avg_df=pd.DataFrame()

for onekeynameip in tqdm(datahj.keys()):
    this_data_close_mean = datahj[onekeynameip]['Close'].mean()
    this_data_range=0
    if(this_data_close_mean<10 and this_data_close_mean>0):
        this_data_range=1
    elif(this_data_close_mean<100 and this_data_close_mean>10):
        this_data_range=2
    elif(this_data_close_mean<1000 and this_data_close_mean>100):
        this_data_range=3
    elif(this_data_close_mean<10000 and this_data_close_mean>1000):
        this_data_range=4
    else:
        this_data_range=5
        
#     print('Mean : ',this_data_close_mean)
#     print('Range : ', this_data_range)
#     print()
    avg_df= avg_df.append({
        
        'Symbol':[datahj[onekeynameip]['Symbol'][0]],
        'close_mean':this_data_close_mean,
        'data_range':this_data_range
        
    }
        , ignore_index=True)
    
    
print(avg_df)

# Index(['Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
#    'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
#    '%Deliverble'],

In [ ]:
range_1_df=avg_df.loc[avg_df['data_range'] == 1]
range_2_df=avg_df.loc[avg_df['data_range'] == 2]
range_3_df=avg_df.loc[avg_df['data_range'] == 3]
range_4_df=avg_df.loc[avg_df['data_range'] == 4]

# print(range_1_df)

range_1_train_data={}
range_2_train_data={}
range_3_train_data={}
range_4_train_data={}

for onerangedatakey in tqdm(range_1_df['Symbol'].to_list()):
    try:
        range_1_train_data[onerangedatakey[0]]=datahj[onerangedatakey[0]]
    except KeyError:
        continue

for onerangedatakey in tqdm(range_2_df['Symbol'].to_list()):
    try:
        range_2_train_data[onerangedatakey[0]]=datahj[onerangedatakey[0]]
    except KeyError:
        continue

for onerangedatakey in tqdm(range_3_df['Symbol'].to_list()):
    try:
        range_3_train_data[onerangedatakey[0]]=datahj[onerangedatakey[0]]
    except KeyError:
        continue

for onerangedatakey in tqdm(range_4_df['Symbol'].to_list()):
    try:
        range_4_train_data[onerangedatakey[0]]=datahj[onerangedatakey[0]]
    except KeyError:
        continue
    
# print(range_1_train_data.keys())
# print()
# print(range_2_train_data.keys())
# print()
# print(range_3_train_data.keys())
# print()
# print(range_4_train_data.keys())
# print()

In [14]:
datahj=range_3_train_data

close_column_data={}
for onestockname in tqdm(datahj.keys()):
    close_column_data[onestockname]=datahj[onestockname].filter(['Close']).values

NameError: name 'range_3_train_data' is not defined

In [ ]:
from pandas import Series

scaled_dataset={}

scale_dict={}

print(len(datahj.keys()))

for onestockname in tqdm(datahj.keys()):
    try:
        ipval_close_column=datahj[onestockname]['Close']
        series = Series(ipval_close_column)
        values = series.values
        values = values.reshape((len(values), 1))
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler=scaler.fit(values)
        scale_dict[onestockname]=scaler
        scaled_dataset[onestockname]  = scaler.transform( close_column_data[onestockname])
    except ValueError:
        print( onestockname +' Value error occured')
        empty_stocks_list.append(onestockname)
        
# with open('scale_dict.p', 'wb') as fp:
#     pickle.dump(scale_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

print(len(scaled_dataset.keys()))

empty_stocks_list=[]     
for deletename in empty_stocks_list:
    try:
        del datahj[onestockname]
    except KeyError:
        print('keyerror ')

In [ ]:
train_lengths_dict={}

for onestocknameipp in tqdm(scaled_dataset.keys()):
    train_lengths_dict[onestocknameipp]=  math.ceil(len(scaled_dataset[onestocknameipp])*0.97)

In [ ]:
x_train = []
y_train = []

train_data_dict={}

for onestocknameipp in tqdm(scaled_dataset.keys()):
    train_data_dict[onestocknameipp] = scaled_dataset[onestocknameipp][0:train_lengths_dict[onestocknameipp] , :]
    for ii in range(60, len(train_data_dict[onestocknameipp])):
        x_train.append(train_data_dict[onestocknameipp][ii-60:ii, :])
        y_train.append(train_data_dict[onestocknameipp][ii, 0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape)
print(y_train.shape)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
x_train.shape[1:]

In [ ]:
x_train.shape

In [ ]:
x_train.shape[1], x_train.shape[2]

In [ ]:
# with open('x_train.p', 'wb') as fp:
#     pickle.dump(x_train, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('y_train.p', 'wb') as fp:
#     pickle.dump(y_train, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# model = Sequential()
# model.add(LSTM(50, return_sequences=True, input_shape= (60,5), kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= True, kernel_initializer='glorot_normal'))
# model.add(LSTM(50, return_sequences= False, kernel_initializer='glorot_normal'))
# model.add(Dense(60,activation='softmax', kernel_initializer='glorot_normal'))
# model.summary()

In [7]:
# input_shape = x_train.shape[1:]
input_shape=(60,4)

inputs = Input(shape=input_shape)

x = LSTM((50), return_sequences=True, kernel_initializer='glorot_normal')(inputs)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(x)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(y)
# x = tensorflow.keras.layers.add([x, y])
x = tensorflow.keras.activations.tanh(y)
# x =  Dense(40, kernel_initializer='glorot_normal')(x)
x = tensorflow.keras.layers.Dropout(0.2)(x)

x = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(x)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(x)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(y)
# x = tensorflow.keras.layers.add([x, y])
x = tensorflow.keras.activations.tanh(y)
# x =  Dense(40, kernel_initializer='glorot_normal')(x)
x = tensorflow.keras.layers.Dropout(0.2)(x)

x = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(x)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(x)
y = LSTM(50, return_sequences=True, kernel_initializer='glorot_normal')(y)
# x = tensorflow.keras.layers.add([x, y])
x = tensorflow.keras.activations.tanh(y)
# x =  Dense(40, kernel_initializer='glorot_normal')(x)
x = tensorflow.keras.layers.Dropout(0.2)(x)

x =  LSTM(50, return_sequences=False, kernel_initializer='glorot_normal')(x)
x = tensorflow.keras.activations.tanh(x)

outputs = Dense(60, kernel_initializer='glorot_normal')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 60, 4)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 60, 50)            11000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
tf_op_layer_Tanh (TensorFlow [(None, 60, 50)]          0         
_________________________________________________________________
dropout (Dropout)            (None, 60, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 50)            20200 

In [8]:
# # model.compile(optimizer='adam', loss='mean_squared_error',metrics=["accuracy"])
model.compile(optimizer=tensorflow.keras.optimizers.Adam(), loss='mean_squared_error',metrics=["accuracy"])
# # model.compile(optimizer=keras.optimizers.adam(lr=0.0005, beta_1=0.9, beta_2=0.999,  decay=0.01, amsgrad=True), loss='mean_squared_error',metrics=["accuracy"])

In [20]:
# model.save('/home/hemangjoshi37a_gmail_com/stkwise_models/TATASTEEL/TATASTEEL_182.h5')
model.save('/home/hemangjoshi37a_gmail_com/stkwise_models60/TATASTEEL/TATASTEEL_26.h5')

In [9]:
homediraddr='/home/hemangjoshi37a_gmail_com'

def print_dim_wise_plots():
    weights_model_ip= model.get_weights()

    list_of_shapes=[]

    for oneweightnumberip in range(len(weights_model_ip)):
        if(weights_model_ip[oneweightnumberip].shape not in list_of_shapes):
            list_of_shapes.append(weights_model_ip[oneweightnumberip].shape)

    shape_wise_weight_dict={}

    for oneweightnumberip in range(len(weights_model_ip)):
        for oneshape in list_of_shapes:
            if(oneshape == weights_model_ip[oneweightnumberip].shape):
                if oneshape not in shape_wise_weight_dict.keys():
                    shape_wise_weight_dict[oneshape]=pd.DataFrame()
                shape_wise_weight_dict[oneshape]=shape_wise_weight_dict[oneshape].append(pd.DataFrame(weights_model_ip[oneweightnumberip]))

    types_of_plot=['line','hist','box','density','area','pie','scatter','hexbin']
    plt.figure()
    for onetypeofplot in types_of_plot:
        for oneshapeofweights in shape_wise_weight_dict.keys():
            try:
                shape_wise_weight_dict[oneshapeofweights].plot(legend=False,figsize=(12,5), 
                                                               kind=onetypeofplot);
                dirlist1= os.listdir(homediraddr+'/figs/png/')
                dirpathip=''
                if(str(onetypeofplot)+'_'+ str(oneshapeofweights) not in dirlist1):
                    os.mkdir(homediraddr+'/figs/png/'+str(onetypeofplot)+'_'+ str(oneshapeofweights)) 
                dirpathip=homediraddr+'/figs/png/'+str(onetypeofplot)+'_'+ str(oneshapeofweights)+'/'
                
                filenum=0
                filecheckpath = dirpathip+str(onetypeofplot)+'_'+ str(oneshapeofweights)+'_'+str(filenum)+'.png'
                while os.path.isfile(filecheckpath):
                    filenum+=1
                    filecheckpath =  dirpathip+str(onetypeofplot)+'_'+ str(oneshapeofweights)+'_'+str(filenum)+'.png'
                
                plt.savefig(dirpathip+str(onetypeofplot)+'_'+ str(oneshapeofweights)+'_'+str(filenum)+".png",
                            dpi=100, bbox_inches='tight')
            except ValueError:
                continue


In [10]:
#plot all the models
class MyCustomCallback(tf.keras.callbacks.Callback):
    counter1=0
#   def on_train_batch_begin(self, batch, logs=None):
    def on_train_batch_end(self, batch, logs=None):
#         clear_output(wait=True)
        
        global counter1
        counter1+=1
        if(counter1%500==0):
            plot_predict()
#         livelossplot.PlotLossesKeras().on_train_batch_end(batch)
        
        
        if(counter1%2500==0):
            print_dim_wise_plots()
            
#             model_weights_df=pd.DataFrame()
#             weights_model= model.get_weights()
#             for onelayerweight in weights_model:
#                 try:
#                     model_weights_df=model_weights_df.append(pd.DataFrame(onelayerweight))
#                 except TypeError:
#                     continue

    #         filenum=0
    #         filecheckpath = '/floyd/home/model_csv/live2/csv/'+str(filenum)+'.csv'
    #         while os.path.isfile(filecheckpath):
    #             filenum+=1
    #             filecheckpath = '/floyd/home/model_csv/live2/csv/'+str(filenum)+'.csv'
    #         model_weights_df.to_csv('/floyd/home/model_csv2/live/csv/' + str(filenum) +  ".csv")


        if(counter1%2500==0):
            filenum=0
            filecheckpath =homediraddr+'/figs/models/'+str(filenum)+'.h5'
            while os.path.isfile(filecheckpath):
                filenum+=1
                filecheckpath = homediraddr+'/figs/models/'+str(filenum)+'.h5'
            model.save(homediraddr+'/figs/models/'+str(filenum)+'.h5')


#   def on_test_batch_begin(self, batch, logs=None):
#   def on_test_batch_end(self, batch, logs=None):

counter1=0

In [11]:
# def plot_predict():
#     global model
    
#     onemodelnameip='TATASTEEL'
#     apple_quote=datahj['TATASTEEL']
#     new_df = apple_quote.filter(['Close'])
    
#     scaler=scale_dict['TATASTEEL']
    
#     last_60_days = new_df[-120:-60].values
#     predicted_plot=last_60_days
#     actual_plot=new_df[-120:].values
#     try:
#         last_60_days_scaled = scaler.transform(last_60_days)
#     except ValueError:
#         _=0
#     for jk in range(60):
#         X_test = []
#         X_test.append(last_60_days_scaled)
#         X_test = np.array(X_test)
#         X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#         pred_price = model.predict(X_test)
#         pred_price_descaled = scaler.inverse_transform(pred_price)
#         predicted_plot=np.append(predicted_plot,pred_price_descaled)
#         array_abcd=np.array(pred_price)
#         asdasdasd=np.append(X_test,array_abcd)
#         asdasdasd=np.delete(asdasdasd, 0,None)
#         last_60_days_scaled = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]
#     plot_df=pd.DataFrame(predicted_plot.T)
#     plot_df2=pd.DataFrame(actual_plot)
#     result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
# #     plt.figure(figsize=(16,8))
# #     plt.autoscale(enable=True, axis='both', tight=None)

#     result.plot(figsize=(11,5))
#     # plt.text(0, 0, onestkstk+' : '+onemodelnameip)
# #     plt.text(0, 0, str(filenum)+' : '+onemodelnameip)

#     filenum=0
#     filecheckpath =homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png'
#     while os.path.isfile(filecheckpath):
#         filenum+=1
#         filecheckpath =homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png'
#     plt.savefig(homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png', dpi=100, bbox_inches='tight')


In [15]:
def plot_predict():
    global model
    
    onemodelnameip='TATASTEEL'
    apple_quote=df
    new_df = apple_quote.filter(['Close'])
    
    scaler=scale_dict['TATASTEEL']
    
    last_60_days = new_df[-180:-120].values
    predicted_plot=last_60_days
    actual_plot=new_df[-180:].values
    try:
        last_60_days_scaled = scaler.transform(last_60_days)
    except ValueError:
        _=0
    for jk in range(120):
        X_test = []
        X_test.append(last_60_days_scaled)
        X_test = np.array(X_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        pred_price = model.predict(X_test)
        pred_price_descaled = scaler.inverse_transform(pred_price)
        predicted_plot=np.append(predicted_plot,pred_price_descaled)
        array_abcd=np.array(pred_price)
        asdasdasd=np.append(X_test,array_abcd)
        asdasdasd=np.delete(asdasdasd, 0,None)
        last_60_days_scaled = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]
    plot_df=pd.DataFrame(predicted_plot.T)
    plot_df2=pd.DataFrame(actual_plot)
    result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
#     plt.figure(figsize=(16,8))
#     plt.autoscale(enable=True, axis='both', tight=None)

    result.plot(figsize=(11,5));
    # plt.text(0, 0, onestkstk+' : '+onemodelnameip)
#     plt.text(0, 0, str(filenum)+' : '+onemodelnameip)

    filenum=0
    filecheckpath =homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png'
    while os.path.isfile(filecheckpath):
        filenum+=1
        filecheckpath =homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png'
    plt.savefig(homediraddr+'/figs/plots/'+'TATASTEEL'+'_'+str(filenum)+'.png', dpi=100, bbox_inches='tight')


In [16]:
plot_predict()
history=model.fit(x_train,
                  y_train,
                  validation_split=0.03,
                  verbose=1,
                  batch_size=1, 
                  epochs=1,
                  callbacks=[
                             livelossplot.PlotLossesKeras(),
#                               wandb.keras.WandbCallback(),
                             MyCustomCallback()
                            ])
plot_predict()
model.save('current35_TATASTEEL_alldata.h5')

UnboundLocalError: local variable 'last_60_days_scaled' referenced before assignment

In [ ]:
#list all model names to iterate and load

from os import listdir
path = '/floyd/home'
def list_files1(directory, extension):
    return (f for f in listdir(directory) if f.endswith('.' + extension))
directory = path
files = list_files1(directory, "h5")

In [ ]:
#plot all the models

types_of_plot=['line','hist','box','kde','density','area','pie','scatter','hexbin']

for onemodelname in tqdm(files):
    model=load_model(onemodelname)
    
    model_weights_df=pd.DataFrame()
    weights_model= model.get_weights()
    for onelayerweight in weights_model:
        try:
            model_weights_df=model_weights_df.append(pd.DataFrame(onelayerweight))
        except TypeError:
            print('Type Error in weights : '+onelayerweight)
            continue

    model_weights_df.to_csv('/floyd/home/model_csv/csv/'+ onemodelname +  ".csv")
    for onetypeofplot in types_of_plot:
        try:
            model_weights_df.plot(legend=False,figsize=(10,5), kind=onetypeofplot)
            plt.savefig('/floyd/home/model_csv/png/'+onetypeofplot+'_'+ onemodelname+".png",
                        dpi=150, bbox_inches='tight')
        except ValueError:
            print('Value Error on plot : '+onetypeofplot+ ' : ' +onemodelname)
            continue

In [ ]:
model.save('resnet_3_45.h5')
model.save(os.path.join(wandb.run.dir, "model.h5"))

In [ ]:
filenum=0
filecheckpath = '/home/hemang/saved_models/'+'saved_model'+str(filenum)+'.h5'

while os.path.isfile(filecheckpath):
    filenum+=1
    filecheckpath = '/home/hemang/saved_models/'+'saved_model'+str(filenum)+'.h5'

model.save(filecheckpath)
print(filecheckpath)

In [ ]:
# GET ALL MODELS

import os
dirlist= os.listdir('/home/hemang/wandb/')
# print(dirlist)
existing_list=[]
for x in os.listdir('/home/hemang/wandb/'):
    if(x!='settings'):
        filepathip='/home/hemang/wandb/'+x+'/model.h5'
        if os.path.isfile(filepathip):
            existing_list.append(filepathip)

print(existing_list)

In [ ]:
#Analyzer Function

def analyze_model_performance(valid,one_model_path,stock_name):
    accuracy=0
    predicted_ok=0
    predicted_notok=0
    total_inc_pred=0
    total_inc_close=0
    total_dnc_pred=0
    total_dnc_close=0
    total_diff_INC=0
    total_diff_DEC=0
    percent_error_INC=0
    percent_error_DEC=0

    for iii in range(len(valid)-1):
        temp_predict_inc=0
        temp_close_inc=0
        if(valid['Predictions'][iii]>valid['Predictions'][iii+1]):
            temp_predict_inc=0
            total_dnc_pred+=valid['Predictions'][iii+1]-valid['Predictions'][iii]
        else:
            temp_predict_inc=1
            total_inc_pred+=valid['Predictions'][iii+1]-valid['Predictions'][iii]

        if(valid['Close'][iii]>valid['Close'][iii+1]):
            temp_close_inc=0
            total_dnc_close+=valid['Close'][iii+1]-valid['Close'][iii]
        else:
            temp_close_inc=1
            total_inc_close+=valid['Close'][iii+1]-valid['Close'][iii]

        if(temp_close_inc==temp_predict_inc):
            predicted_ok+=1
        else:
            predicted_notok+=1

        
    total_diff_INC=total_inc_pred-total_inc_close
    total_diff_DEC=total_dnc_pred-total_dnc_close
    
    if(total_inc_close!=0):
        percent_error_INC=100*(total_inc_pred-total_inc_close)/total_inc_close
    else:
        percent_error_INC='INF'
    
    if(total_dnc_close!=0):
        percent_error_DEC=100*(total_dnc_pred-total_dnc_close)/total_dnc_close
    else:
        percent_error_DEC='INF'
    
    accuracy=100*predicted_ok/(predicted_ok+predicted_notok)
    
    return {
        'accuracy':accuracy,
        'predicted_ok' :   predicted_ok,
        'predicted_notok':predicted_notok,
        'total_inc_pred':total_inc_pred,
        'total_inc_close':total_inc_close,
        'total_dnc_pred':total_dnc_pred,
        'total_dnc_close':total_dnc_close,
        'total_diff_INC':total_diff_INC,
        'total_diff_DEC':total_diff_DEC,
        'percent_error_INC':percent_error_INC,
        'percent_error_DEC':percent_error_DEC,
        'valid_mean_diff':[valid.mean()['diff']],
        'valid_max_diff':[valid.max()['diff']],
        'valid_min_diff':[valid.min()['diff']],
        'path':one_model_path,
        'stock_name':stock_name,
    }
#Analyzer Function

In [ ]:
#Model Wise Analysis

perforamnce_table = {
        'accuracy':[],
        'stock_name':[],
        'path':[],
        'valid_mean_diff':[],
        'valid_max_diff':[],
        'valid_min_diff':[],
        'predicted_ok' :[],
        'predicted_notok':[],
        'total_inc_pred':[],
        'total_inc_close':[],
        'total_dnc_pred':[],
        'total_dnc_close':[],
        'total_diff_INC':[],
        'total_diff_DEC':[],
        'percent_error_INC':[],
        'percent_error_DEC':[]
}

perforamnce_table_df=pd.DataFrame(perforamnce_table)

for one_model_path in existing_list:
    try:
        model = load_model(one_model_path)

        predictions = model.predict(x_test)
        predictions = scaler.inverse_transform(predictions)

        train = data[:training_data_len]
        valid = data[training_data_len:]
        valid['Predictions'] = predictions
        valid['diff']=valid['Close']-valid['Predictions']
        valid=valid[['Close', 'Predictions','diff']]

        analyzeddataget= pd.DataFrame(analyze_model_performance(valid,one_model_path))
        
        perforamnce_table_df=perforamnce_table_df.append(analyzeddataget,ignore_index=True)

    except ValueError:
        print('Value error found')

print(perforamnce_table_df)
perforamnce_table_df.to_csv (r'/home/hemang/model_analysis.csv', index = False, header=True)

In [ ]:
#Stockwise Analysis

perforamnce_table2 = {
        'accuracy':[],
        'stock_name':[],
        'path':[],
        'valid_mean_diff':[],
        'valid_max_diff':[],
        'valid_min_diff':[],
        'predicted_ok' :[],
        'predicted_notok':[],
        'total_inc_pred':[],
        'total_inc_close':[],
        'total_dnc_pred':[],
        'total_dnc_close':[],
        'total_diff_INC':[],
        'total_diff_DEC':[],
        'percent_error_INC':[],
        'percent_error_DEC':[]
}
perforamnce_table_df2=pd.DataFrame(perforamnce_table2)

for onestockippp in datahj.keys():
    print(onestockippp)
    try:
        df5 = datahj[onestockippp]
        data3 = df5.filter(['Close'])  #old_line

        dataset = data3.values
        training_data_len = math.ceil( len(dataset) * .97 )

        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_data = scaler.fit_transform(dataset)

        x_train3 = []
        y_train3 = []
        train_data = scaled_data[0:training_data_len , :]
        for i in range(60, len(train_data)):
          x_train3.append(train_data[i-60:i, :])
          y_train3.append(train_data[i, 0])

        x_train3, y_train3 = np.array(x_train3), np.array(y_train3)

        test_data = scaled_data[training_data_len - 60: , :]
        x_test3 = []
        y_test3 = dataset[training_data_len:, :]
        for i in range(60, len(test_data)):
          x_test3.append(test_data[i-60:i, :])

        x_test3 = np.array(x_test3)

        x_test3 = np.reshape(x_test3, (x_test3.shape[0], x_test3.shape[1], 1 ))
        x_test3.shape


        ###########################################


        predictions = model.predict(x_test3)
        predictions = scaler.inverse_transform(predictions)

        train = data3[:training_data_len]
        valid = data3[training_data_len:]
        valid['Predictions'] = predictions

        valid['diff']=valid['Close']-valid['Predictions']
        valid=valid[['Close', 'Predictions','diff']]
        one_model_path='/home/hemang/wandb/run-20200407_125141-29n1reyw/model.h5'

        analyzeddataget= pd.DataFrame(analyze_model_performance(valid,one_model_path,onestockippp))

#         print(onestockippp , ' =:= ',analyzeddataget)

        perforamnce_table_df2=perforamnce_table_df2.append(analyzeddataget,ignore_index=True)

#         print('-------------------------------------------------------------------------------------------------')
        
    except ValueError:
        print('Value Error Found in stockwise.')

    
print(perforamnce_table_df2)
perforamnce_table_df2.to_csv (r'/home/hemang/model_analysis_sharewise.csv', index = False, header=True)

In [ ]:
#Stock and Model Wise analysis 
pd.options.mode.chained_assignment = None  # default='warn'
perforamnce_table = {
        'accuracy':[],
        'stock_name':[],
        'path':[],
        'valid_mean_diff':[],
        'valid_max_diff':[],
        'valid_min_diff':[],
        'predicted_ok' :[],
        'predicted_notok':[],
        'total_inc_pred':[],
        'total_inc_close':[],
        'total_dnc_pred':[],
        'total_dnc_close':[],
        'total_diff_INC':[],
        'total_diff_DEC':[],
        'percent_error_INC':[],
        'percent_error_DEC':[]
}

perforamnce_table_df=pd.DataFrame(perforamnce_table)
valuerrrorstoccks=[]
valueerrormodel=[]

for one_model_path in tqdm(existing_list):
    model = load_model(one_model_path)
    for onestockippp in datahj.keys():
        try:
            df5 = datahj[onestockippp]
            data = df5.filter(['Close'])  #old_line

            dataset = data.values
            training_data_len = math.ceil( len(dataset) * .97 )

            scaler = MinMaxScaler(feature_range=(0,1))
            scaled_data = scaler.fit_transform(dataset)

            x_train = []
            y_train = []
            train_data = scaled_data[0:training_data_len , :]
            for i in range(60, len(train_data)):
              x_train.append(train_data[i-60:i, :])
              y_train.append(train_data[i, 0])

            x_train, y_train = np.array(x_train), np.array(y_train)

            test_data = scaled_data[training_data_len - 60: , :]
            x_test = []
            y_test = dataset[training_data_len:, :]
            for i in range(60, len(test_data)):
              x_test.append(test_data[i-60:i, :])

            x_test = np.array(x_test)

            x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

            predictions = model.predict(x_test)
            predictions = scaler.inverse_transform(predictions)
            train = data[:training_data_len]
            valid = data[training_data_len:]
            valid['Predictions'] = predictions
            valid['diff']=valid['Close']-valid['Predictions']
            valid=valid[['Close', 'Predictions','diff']]
            analyzeddataget= pd.DataFrame(analyze_model_performance(valid,one_model_path,onestockippp))
            perforamnce_table_df=perforamnce_table_df.append(analyzeddataget,ignore_index=True)

        except ValueError:
            continue

print(perforamnce_table_df)
perforamnce_table_df.to_csv (r'/home/hemang/model_analysis_all.csv', index = False, header=True)  

In [ ]:
#final performance matrix evaluation using stock&model_wise_analysis_data

perforamnce_table_df = pd.read_csv("model_analysis_all.csv") 

# only_path_and_stkname=perforamnce_table_df

perforamnce_table_df=perforamnce_table_df.replace([np.inf, -np.inf], np.nan)

new_scaled_df=perforamnce_table_df

new_scaled_df['valid_mean_diff']= perforamnce_table_df['valid_mean_diff']/ max([abs(perforamnce_table_df.min()['valid_mean_diff']),abs(perforamnce_table_df.max()['valid_mean_diff'])])
new_scaled_df['valid_mean_diff']=new_scaled_df['valid_mean_diff'].abs()

new_scaled_df['valid_min_diff']= perforamnce_table_df['valid_min_diff']/ max([abs(perforamnce_table_df.min()['valid_min_diff']),abs(perforamnce_table_df.max()['valid_min_diff'])])
new_scaled_df['valid_min_diff']=new_scaled_df['valid_min_diff'].abs()

new_scaled_df['valid_max_diff']= perforamnce_table_df['valid_max_diff']/ max([abs(perforamnce_table_df.min()['valid_max_diff']),abs(perforamnce_table_df.max()['valid_max_diff'])])
new_scaled_df['valid_max_diff']=new_scaled_df['valid_max_diff'].abs()

new_scaled_df['predicted_ok']= 1-( perforamnce_table_df['predicted_ok']/perforamnce_table_df.max()['predicted_ok'])
new_scaled_df['predicted_notok']=  perforamnce_table_df['predicted_notok']/perforamnce_table_df.max()['predicted_notok']

new_scaled_df['accuracy']=1-(perforamnce_table_df['accuracy']/100)

new_scaled_df['total_inc_pred']=perforamnce_table_df['total_inc_pred']/perforamnce_table_df.max()['total_inc_pred']
new_scaled_df['total_dnc_pred']=perforamnce_table_df['total_dnc_pred']/perforamnce_table_df.min()['total_dnc_pred']

new_scaled_df=new_scaled_df.drop(['total_dnc_close', 'total_inc_close'], axis=1)

new_scaled_df['total_diff_INC']= perforamnce_table_df['total_diff_INC']/ max([abs(perforamnce_table_df.min()['total_diff_INC']),abs(perforamnce_table_df.max()['total_diff_INC'])])
new_scaled_df['total_diff_INC']=new_scaled_df['total_diff_INC'].abs()

new_scaled_df['total_diff_DEC']= perforamnce_table_df['total_diff_DEC']/ max([abs(perforamnce_table_df.min()['total_diff_DEC']),abs(perforamnce_table_df.max()['total_diff_DEC'])])
new_scaled_df['total_diff_DEC']=new_scaled_df['total_diff_DEC'].abs()

new_scaled_df['percent_error_INC']= perforamnce_table_df['percent_error_INC']/ max([abs(perforamnce_table_df.min()['percent_error_INC']),abs(perforamnce_table_df.max()['percent_error_INC'])])
new_scaled_df['percent_error_INC']=new_scaled_df['percent_error_INC'].abs()

new_scaled_df['percent_error_DEC']= perforamnce_table_df['percent_error_DEC']/ max([abs(perforamnce_table_df.min()['percent_error_DEC']),abs(perforamnce_table_df.max()['percent_error_DEC'])])
new_scaled_df['percent_error_DEC']=new_scaled_df['percent_error_DEC'].abs()

stk_path_drop=new_scaled_df.drop(['stock_name', 'path'], axis=1)
stk_path_drop
stk_path_drop["sum"] = stk_path_drop.sum(axis=1)

stk_path_drop['path']=perforamnce_table_df['path']
stk_path_drop['stock_name']=perforamnce_table_df['stock_name']

print(stk_path_drop)

stk_path_drop.to_csv (r'/home/hemang/full_final.csv', index = False, header=True)  
# print('MIN /n',stk_path_drop.min())
# print('MAX /n',stk_path_drop.max())
# print('MEAN /n',stk_path_drop.mean())

In [ ]:
#check modelwise accuracy

for klkl in tqdm(perforamnce_table_df.index):
    
model_wise_accuracy=[]
for onemodelnn in existing_list:
    onemodeldata=perforamnce_table_df.loc[perforamnce_table_df['path'] == onemodelnn]
    onemodeldata.mean(axis = 0)['']

In [ ]:
#Create the testing data set
#Create a new array containing scaled values from index 1543 to 2002 

x_test = []
y_test = []

# test_data_dict={}
# for onestocknameipp in tqdm(scaled_dataset.keys()):
#     test_data_dict[onestocknameipp] = scaled_dataset[onestocknameipp][0:train_lengths_dict[onestocknameipp] , :]
#     y_test = scaled_dataset[onestocknameipp][train_lengths_dict[onestocknameipp]:, :]
#     for ii in range(60, train_lengths_dict[onestocknameipp]):
#         x_test.append(test_data_dict[onestocknameipp][ii-60:ii, :])

test_data = scaled_data[training_data_len - 60: , :]
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, :])

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
x_test.shape

predictions = model.predictions(x_test)
predictions = scaler.inverse_transform(predictions)
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

# train_ipp=close_column_data['MTNL'][:train_lengths_dict['MTNL']]
# valid_ipp=close_column_data['MTNL'][train_lengths_dict['MTNL']:]
# valid_ipp['Predictions'] = predictions

#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

#Show the valid and predicted prices
valid['diff']=valid['Close']-valid['Predictions']
valid=valid[['Close', 'Predictions','diff']]
valid.plot(figsize=(16,8))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cm = metrics.confusion_matrix(test_batch.classes, y_pred)
# or
#cm = np.array([[1401,    0],[1112, 0]])

plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.xticks([], [])
plt.yticks([], [])
plt.title('Confusion matrix ')
plt.colorbar()
plt.show()

In [ ]:
#Get the quote
stock_name='NIFTY50'
apple_quote =nsepy.get_history(symbol=stock_name, start=date(2012,1,1), end=date.today())
#Create a new dataframe
model=load_model('/home/hemang/wandb/run-20200408_112313-r5pghjhu/model.h5')
new_df = apple_quote.filter(['Close'])
#Get teh last 60 day closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
last_60_days_scaled = scaler.fit_transform(last_60_days)
#Create an empty list
X_test = []
#Append teh past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling 
pred_price = scaler.inverse_transform(pred_price)
print(pred_price[0],type(pred_price))

In [ ]:
#Get the quote
apple_quote2 =nsepy.get_history(symbol=stock_name, start=date(2012,1,1), end=date.today())
print(apple_quote2['Close'])

In [ ]:
import datetime
# predict_stock_list=['GAYAPROJ','JPPOWER','SIMPLEXINF','GREENPOWER','SICAL','HDIL','AKSHOPTFBR','COX&KINGS','ALOKTEXT','FLEXITUFF']
# predict_stock_list=newlsstttt
# stock_get_dict_data={}
# ipstockdata={}
sixtydaysago = date.today()-datetime.timedelta(days = 500)

# for ippstockname in tqdm(predict_stock_list):
#     try:
#         ipstockdata[ippstockname]=nsepy.get_history(symbol=ippstockname, start=sixtydaysago, end=date.today())
#     except ValueError:
#         print('Value error occured in ',ippstockname)
#         continue
#     except KeyError:
#         print("age is unknown. c1")
#     except SSLError as e:
#         print('SSL error uccured. c1')
#     except ConnectionError as e:
#         print('connection error. c1')
#     except ConnectionResetError as e:
#         print('connection reset error. c1')
#     except AttributeError:
#         print('attributre eror accured. c1')
#     except RemoteDisconnected:
#         print('Remote disconnected error c1')
#     except ProtocolError:
#         print('protocol error occured c1')
        
# print(ipstockdata)

# with open('two_hun_stk_data.p', 'wb') as fp:
#     pickle.dump(ipstockdata, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open('/kaggle/input/datasetip/two_hun_stk_data.p', 'rb') as fp:
    ipstockdata = pickle.load(fp)


In [ ]:
#Remove empty things
removables2=[]
for onename in ipstockdata.keys():
    nparrayy=np.array(ipstockdata[onename])
    if(nparrayy.size==0):
        removables2.append(onename)
        
for oneremovename in removables2:
    del ipstockdata[oneremovename]
    
print(removables2)
print(len(removables2))

In [ ]:
#stock prediction list wise and model wise

predicteddataframe=pd.DataFrame({'stock_name':[],
                        'model':[],
                        'today_date': [],
                        'predicted_price':[],
                        'index':[]
                      })

for onepathname in tqdm(existing_list):

    model = load_model(onepathname)
    print(onepathname)
    indexhjhj=0
    model = load_model(onepathname)

    for onnnestknaeemmeee in ipstockdata.keys():
        try:
            apple_quote=ipstockdata[onnnestknaeemmeee]
            new_df = apple_quote.filter(['Close'])
            last_60_days = new_df[-60:].values
            scaler = MinMaxScaler(feature_range=(0,1))
            last_60_days_scaled = scaler.fit_transform(last_60_days)
            X_test = []
            X_test.append(last_60_days_scaled)
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#             print(onnnestknaeemmeee, '----',X_test)
#             print(onnnestknaeemmeee, '----',X_test.shape)
            pred_price = model.predict(X_test)
            pred_price = scaler.inverse_transform(pred_price)
            data_dict={'stock_name':onnnestknaeemmeee,
                        'model':onepathname,
                        'today_date': date.today(),
                        'predicted_price':pred_price[0],
                       'index':indexhjhj
                      }
            indexhjhj+=1

            predicteddataframe=predicteddataframe.append(pd.DataFrame(data_dict))
    
        except ValueError as dede:

            print('Value error occured.',dede)
            continue
#         except KeyError:
#             print("age is unknown.")
        except SSLError as e:
            print('SSL error uccured.')
        except ConnectionError as e:
            print('connection error.')
        except ConnectionResetError as e:
            print('connection reset error.')
        except AttributeError:
            print('attributre eror accured.')
        except RemoteDisconnected:
            print('Remote disconnected error')
        except ProtocolError:
            print('protocol error occured')


predicteddataframe
predicteddataframe.to_excel('two_hunder_predicted.xlsx')

In [ ]:
#clusyerize results

points=predicteddataframe.loc[predicteddataframe['stock_name'] == 'ADHUNIK']['predicted_price']
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
points_array=points.to_numpy()
reshapen=points_array.reshape(-1,1)
kmeans.fit(reshapen)
print(kmeans.cluster_centers_)
y_km = kmeans.fit_predict(reshapen)
plt.scatter(reshapen[y_km ==0,0], reshapen[y_km == 0,0], s=100, c='red')
plt.scatter(reshapen[y_km ==1,0], reshapen[y_km == 1,0], s=100, c='black')
plt.scatter(reshapen[y_km ==2,0], reshapen[y_km == 2,0], s=100, c='blue')
# plt.scatter(reshapen[y_km ==3,0], reshapen[y_km == 3,0], s=100, c='cyan')

In [ ]:
#stock prediction list wise and model wise

predicteddataframe=pd.DataFrame({'stock_name':[],
                        'model':[],
                        'today_date': [],
                        'predicted_price':[],
                        'index':[]
                      })

prediction_series={}

for onepathname in tqdm(existing_list):

    model = load_model(onepathname)
    print(onepathname)
#     model = load_model(onepathname)

    for onnnestknaeemmeee in ipstockdata.keys():
        try:
            apple_quote=ipstockdata[onnnestknaeemmeee]
            new_df = apple_quote.filter(['Close'])
            last_60_days = new_df[-60:].values
            scaler = MinMaxScaler(feature_range=(0,1))
            last_60_days_scaled = scaler.fit_transform(last_60_days)
            X_test = []
            X_test.append(last_60_days_scaled)
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            pred_price = model.predict(X_test)
            pred_price = scaler.inverse_transform(pred_price)
            data_dict={'stock_name':onnnestknaeemmeee,
                        'model':onepathname,
                        'today_date': date.today(),
                        'predicted_price':pred_price[0],
                      }

            predicteddataframe=predicteddataframe.append(pd.DataFrame(data_dict))
    
        except ValueError as dede:
            print('Value error occured.',dede)
            continue
        except SSLError as e:
            print('SSL error uccured.')
        except ConnectionError as e:
            print('connection error.')
        except ConnectionResetError as e:
            print('connection reset error.')
        except AttributeError:
            print('attributre eror accured.')
        except RemoteDisconnected:
            print('Remote disconnected error')
        except ProtocolError:
            print('protocol error occured')


predicteddataframe
predicteddataframe.to_excel('two_hunder_predicted.xlsx')

In [ ]:
model_dict={}
for onemodelnameip in  tqdm(existing_list):
    model_dict[onemodelnameip]= load_model(onemodelnameip)
# model_dict

In [ ]:
stk_name112='TATASTEEL'
sixtydaysago2 = date.today()-datetime.timedelta(days = 500)
apple_quote=nsepy.get_history(symbol=stk_name112, start=sixtydaysago2, end=date.today())

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
for onestkstk in tqdm(ipstockdata.keys()):
    model_num=0
    apple_quote=ipstockdata[onestkstk]
    for onemodelnameip in  tqdm(model_dict.keys()):
        model = model_dict[onemodelnameip]
        new_df = apple_quote.filter(['Close'])
        last_60_days = new_df[-120:-60].values
        predicted_plot=last_60_days
        actual_plot=new_df[-120:].values

        for jk in range(60):
            try:
                last_60_days_scaled = scaler.fit_transform(last_60_days)
            except ValueError:
                continue   
            X_test = []
            X_test.append(last_60_days_scaled)
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            try:
                pred_price = model.predict(X_test)
            except ValueError:
                continue
            pred_price_descaled = scaler.inverse_transform(pred_price)
            predicted_plot=np.append(predicted_plot,pred_price_descaled)
            array_abcd=np.array(pred_price_descaled)
            asdasdasd=np.append(X_test,array_abcd)
            asdasdasd=np.delete(asdasdasd, 0,None)
            last_60_days = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]

        plot_df=pd.DataFrame(predicted_plot.T)
        plot_df2=pd.DataFrame(actual_plot)
        result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
        plt.figure(figsize=(16,8))
        plt.autoscale(enable=True, axis='both', tight=None)
        
        result.plot(figsize=(16,8))
        plt.text(0, 0, onestkstk+' : '+onemodelnameip)
        plt.savefig('/home/hemang/figs/'+onestkstk+str(model_num)+'.png', dpi=200, bbox_inches='tight')
        model_num+=1


In [ ]:
#Model Wise plotter

modelnameinto='/kaggle/input/resnet-models/resnet_2_glorot_uni_20_layer_deep.h'

model = load_model(modelnameinto)

ipstockdata=range_2_train_data

scaler = MinMaxScaler(feature_range=(0,5))
for onestkstk in tqdm(ipstockdata.keys()):
    model_num=0
    onemodelnameip=modelnameinto
    apple_quote=ipstockdata[onestkstk]
#     model = load_model('model11225.h')
    new_df = apple_quote.filter(['Close'])
    last_60_days = new_df[-120:-60].values
    predicted_plot=last_60_days
    actual_plot=new_df[-120:].values
    for jk in range(60):
        try:
            last_60_days_scaled = scaler.fit_transform(last_60_days)
        except ValueError:
            continue   
        X_test = []
        X_test.append(last_60_days_scaled)
        X_test = np.array(X_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        try:
            pred_price = model.predict(X_test)
        except ValueError:
            continue
        pred_price_descaled = scaler.inverse_transform(pred_price)
        predicted_plot=np.append(predicted_plot,pred_price_descaled)
        array_abcd=np.array(pred_price_descaled)
        asdasdasd=np.append(X_test,array_abcd)
        asdasdasd=np.delete(asdasdasd, 0,None)
        last_60_days = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]

    plot_df=pd.DataFrame(predicted_plot.T)
    plot_df2=pd.DataFrame(actual_plot)
    result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
    plt.figure(figsize=(16,8))
    plt.autoscale(enable=True, axis='both', tight=None)

    result.plot(figsize=(16,8))
    plt.text(0, 0, onestkstk+' : '+onemodelnameip)
    try:
        plt.savefig('fig10/'+onestkstk+str(model_num)+'.png', dpi=200, bbox_inches='tight')
    except ValueError:
        continue
    model_num+=1


In [ ]:
#Model Wise plotter
import gc
modelnameinto='TATASTEEL_MODEL'

ipstockdata=datahj['TATASTEEL']

for model_num  in tqdm(range(94,134)):
    # model_num=0
    clear_output(wait=True)
    model_path_new='/floyd/home/model_csv/live/models/'+str(model_num)+'.h5'
    model=load_model(model_path_new)
    onemodelnameip=modelnameinto
    apple_quote=ipstockdata
    new_df = apple_quote.filter(['Close'])
    last_60_days = new_df[-120:-60].values
    predicted_plot=last_60_days
    actual_plot=new_df[-120:].values
    for jk in range(60):
        try:
            last_60_days_scaled = scaler.transform(last_60_days)
        except ValueError:
            continue   
        X_test = []
        X_test.append(last_60_days_scaled)
        X_test = np.array(X_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        pred_price = model.predict(X_test)
        pred_price_descaled = scaler.inverse_transform(pred_price)
        predicted_plot=np.append(predicted_plot,pred_price_descaled)
        array_abcd=np.array(pred_price_descaled)
        asdasdasd=np.append(X_test,array_abcd)
        asdasdasd=np.delete(asdasdasd, 0,None)
        last_60_days = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]
    plot_df=pd.DataFrame(predicted_plot.T)
    plot_df2=pd.DataFrame(actual_plot)
    result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
#     plt.figure(figsize=(16,8))
#     plt.autoscale(enable=True, axis='both', tight=None)

    result.plot(figsize=(16,8))
    # plt.text(0, 0, onestkstk+' : '+onemodelnameip)
    plt.text(0, 0, str(model_num)+' : '+onemodelnameip)
    plt.savefig('/floyd/home/model_csv/live/plots/'+str(model_num)+'TATASTEEL.png', dpi=200, bbox_inches='tight')
    del model
    gc.collect()

In [ ]:
#!/usr/bin/env python
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('fig17.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/floyd/home/fig4/', zipf)
zipf.close()

In [ ]:
# GIF Convert script in terminal
# convert -delay 20 -loop 0 *.jpg myimage.gif

In [ ]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key="w19o0chuo929jxkp")
kite.login_url()

In [ ]:
data = kite.generate_session("jG2jTauB0gw6u5vZzF6oR1YZpP6I2c7V", api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
kite.set_access_token(data["access_token"])

In [ ]:

import datetime
from datetime import timedelta
delta = timedelta(days=100)

todaydt=datetime.date.today()

hud_ago=todaydt-delta

old_lst=[]

instrument_token='895745'
interval='5minute'
delta1 = timedelta(days=99)
delta = timedelta(days=100)
todaydt=datetime.date.today()
hud_ago=todaydt-delta1

to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)

print(to_date)
print(from_date)

old_lst=kite.historical_data(instrument_token, from_date, to_date, interval,continuous=False)

print(len(old_lst))

In [ ]:
for i2 in tqdm(range(22)):

    todaydt=todaydt-delta
    hud_ago=hud_ago-delta
    
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
    
    new_lst = kite.historical_data(instrument_token, from_date, to_date, interval,continuous=False)
    old_lst = new_lst + old_lst
    
    if(len(new_lst)==0):
        print('empty')

In [ ]:
TATASTEEL_DF=pd.DataFrame(old_lst)
print(len(old_lst))
TATASTEEL_DF


In [ ]:
# inst_all=kite.instruments()

# # {'exchange': 'BCD',
# #   'exchange_token': '2111805',
# #   'expiry': datetime.date(2020, 12, 29),
# #   'instrument_token': 540622086,
# #   'instrument_type': 'PE',
# #   'last_price': 0.0,
# #   'lot_size': 1,
# #   'name': 'GBPINR',
# #   'segment': 'BCD-OPT',
# #   'strike': 93.0,
# #   'tick_size': 0.0025,
# #   'tradingsymbol': 'GBPINR20DEC93.0000PE'},

In [ ]:
# kite.holdings()

In [ ]:
# kite.orders()

In [ ]:
with open('TATASTEEL_DF.p', 'wb') as fp:
    pickle.dump(TATASTEEL_DF, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
TATASTEEL_DF